In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import copy
from torchmetrics.functional import r2_score as torch_r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the data

In [66]:
df = pd.read_csv("/kaggle/input/us-college-completion-rate-analysis/train.csv")

In [67]:
features = ['Tuition_in_state','Tuition_out_state','Faculty_salary','Pell_grant_rate','SAT_average','ACT_50thPercentile','pct_White','pct_Black','pct_Hispanic','pct_Asian','Parents_middlesch','Parents_highsch','Parents_college'] 
X = df[features]
y = df["Completion_rate"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Preprocessing

In [68]:
#Comment out when training
scaler = StandardScaler()
scaler.fit(X)
X_train = scaler.transform(X)

In [ ]:
#Comment out when sending to Kaggle
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [69]:
#Real X_Test
real_X_test = pd.read_csv("/kaggle/input/us-college-completion-rate-analysis/x_test.csv")
real_X_test = real_X_test.drop(columns=['Unnamed: 0'])
real_X_test = scaler.transform(real_X_test)

## RandomForestRegressor regressor (rf)

In [11]:
# rf.get_params() (Initial Parameters)

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

### HyperParameters Tuning

Researching Online, we have identified the following Hyperparameters to improve prediction for the RandomForest Model:
1. n_estimators (Number of trees in random forest)
2. max_features (Number of features to consider at every split)
3. max_depth (Maximum number of levels in tree)
4. min_samples_split (Minimum number of samples required to split a node)
5. min_samples_leaf (Minimum number of samples required at each leaf node)

We then use GridSearchCV to identify the best parameters for each hyperparameter

References: 
MOHIT, Random Forest Hyperparameter tuning. Available: https://www.kaggle.com/code/mohitsital/random-forest-hyperparameter-tuning/notebook

Nisha Arya , Tuning Random Forest Hyperparameters. Available: https://www.kdnuggets.com/2022/08/tuning-random-forest-hyperparameters.html#:~:text=Hyperparameters%20of%20a%20Random%20Forest&text=n_estimators%3A%20This%20is%20the%20number,looking%20for%20the%20best%20split.

### 1st Optimization Run

In [29]:
# randomforest_param_grid = { 
#     'n_estimators': [50,100,200,300],
#     'max_depth' : [2,50,80,120],
#     'min_samples_leaf' :[1,2,4],
#     'min_samples_split':[2,4,6],
#     'max_features' : [1.0, 'sqrt']
    
# }

# rf_tuned = RandomForestRegressor()
# CV_REG = GridSearchCV(estimator=rf_tuned, param_grid=randomforest_param_grid, scoring ='neg_root_mean_squared_error')
# CV_REG.fit(X_train, y_train)
# CV_REG.best_params_

#Output:
# {'max_depth': 50,
#  'max_features': 'sqrt',
#  'min_samples_leaf': 1,
#  'min_samples_split': 2,
#  'n_estimators': 300}

{'max_depth': 50,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

### 2nd Optimization Run

In [39]:
# rf_tuned = RandomForestRegressor(n_estimators=300, random_state=42, max_depth = 50, max_features = 'sqrt',min_samples_leaf= 1, min_samples_split =2)
# rf_tuned.fit(X_train, y_train)
# r_sq_tuned = rf_tuned.score(X_train, y_train)
# print(r_sq_tuned)
# y_pred_tuned = rf_tuned.predict(real_X_test)
# print(y_pred_tuned)
# print(y_pred_tuned.shape)
# print(type(y_pred_tuned))

#Output:
# 0.9806309970187881
# [0.451092   0.46162267 0.479439   ... 0.54139    0.568323   0.64028967]
# (1165,)

0.9806309970187881
[0.451092   0.46162267 0.479439   ... 0.54139    0.568323   0.64028967]
(1165,)
<class 'numpy.ndarray'>


### 3rd Optimization Run

In [13]:
# randomforest_param_grid_v2 = { 
#     'max_depth' : [30,40,50,60,70],
#     'min_samples_leaf' :[1,2,4],
#     'min_samples_split':[2,4,6],
#     'max_features' : [1.0, 'sqrt','log2']
    
# }

# rf_tuned_v2  = RandomForestRegressor()
# CV_REG_v2  = GridSearchCV(estimator=rf_tuned_v2, param_grid=randomforest_param_grid_v2, scoring ='neg_root_mean_squared_error')
# CV_REG_v2.fit(X_train, y_train)
# CV_REG_v2.best_params_

#Output:
# {'max_depth': 30,
#  'max_features': 1.0,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2}

{'max_depth': 30,
 'max_features': 1.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

### 4th Optimization Run

In [14]:
# randomforest_param_grid_v2 = { 
#     'max_depth' : [15,20,25,30,35],
#     'min_samples_leaf' :[1],
#     'min_samples_split':[2],
#     'max_features' : [1.0]
    
# }

# rf_tuned_v2  = RandomForestRegressor()
# CV_REG_v2  = GridSearchCV(estimator=rf_tuned_v2, param_grid=randomforest_param_grid_v2, scoring ='neg_root_mean_squared_error')
# CV_REG_v2.fit(X_train, y_train)
# CV_REG_v2.best_params_

#Output:
# {'max_depth': 15,
#  'max_features': 1.0,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2}

{'max_depth': 15,
 'max_features': 1.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

### 5th Optimization Run

In [16]:
# randomforest_param_grid_v2 = { 
#     'max_depth' : [12,13,14,15,16,17,18],
#     'min_samples_leaf' :[1],
#     'min_samples_split':[2],
#     'max_features' : [1.0]
    
# }

# rf_tuned_v2  = RandomForestRegressor()
# CV_REG_v2  = GridSearchCV(estimator=rf_tuned_v2, param_grid=randomforest_param_grid_v2, scoring ='neg_root_mean_squared_error')
# CV_REG_v2.fit(X_train, y_train)
# CV_REG_v2.best_params_

#Output:
# {'max_depth': 16,
#  'max_features': 1.0,
#  'min_samples_leaf': 1,
#  'min_samples_split': 2}

{'max_depth': 16,
 'max_features': 1.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

## n_estimator hypertuning using 5th Optimization Run

n_estimators=100

In [17]:
# rf_tuned_v3  = RandomForestRegressor(n_estimators = 100,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
# rf_tuned_v3.fit(X_train, y_train)
# r_sq = rf_tuned_v3.score(X_train, y_train)
# print(r_sq)

#0.9791291118239913

0.9791291118239913


In [18]:
# y_pred_TEST = rf_tuned_v3.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST, squared=False)
# RMSE

#0.06752790285476055

0.06752790285476055

n_estimators=200

In [19]:
# rf_tuned_v3  = RandomForestRegressor(n_estimators = 200,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
# rf_tuned_v3.fit(X_train, y_train)
# r_sq = rf_tuned_v3.score(X_train, y_train)
# print(r_sq)

#0.9795947087864706

0.9795947087864706


In [20]:
# y_pred_TEST = rf_tuned_v3.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST, squared=False)
# RMSE

#0.06665104626315474

0.06665104626315474

n_estimators=250

In [38]:
# rf_tuned_v3  = RandomForestRegressor(n_estimators = 250,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
# rf_tuned_v3.fit(X_train, y_train)
# r_sq = rf_tuned_v3.score(X_train, y_train)
# print(r_sq)

#0.9802005813521949

0.9802005813521949


In [39]:
# y_pred_TEST = rf_tuned_v3.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST, squared=False)
# RMSE

#0.06654103031746121

0.06654103031746121

n_estimators=300

In [40]:
# rf_tuned_v3  = RandomForestRegressor(n_estimators = 300,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
# rf_tuned_v3.fit(X_train, y_train)
# r_sq = rf_tuned_v3.score(X_train, y_train)
# print(r_sq)

#0.9800403271293735

0.9800403271293735


In [41]:
# y_pred_TEST = rf_tuned_v3.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST, squared=False)
# RMSE

#0.06701492324537602

0.06701492324537602

n_estimators=400

In [32]:
# rf_tuned_v3  = RandomForestRegressor(n_estimators = 400,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
# rf_tuned_v3.fit(X_train, y_train)
# r_sq = rf_tuned_v3.score(X_train, y_train)
# print(r_sq)

#0.9800626376393404

0.9800626376393404


In [33]:
# y_pred_TEST = rf_tuned_v3.predict(X_test)

# #Testing RMSE error for the Train Test split
# RMSE = mean_squared_error(y_test, y_pred_TEST, squared=False)
# RMSE

#0.06704377902368457

0.06704377902368457

From n_estimators =300 and onwards, the training R-squared score is less than that of the n_estimators=250 model (0.9802005813521949) while the test data RMSE is greater than that of the n_estimators=250 model (0.06654103031746121) and the final model for the RandomForestRegressor model has the following parameters:

n_estimators = 250,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2

## Using Final Model to make prediction for Kaggle Submission

In [70]:
rf_tuned_v3  = RandomForestRegressor(n_estimators = 250,max_depth =16, max_features =1.0, min_samples_leaf=1,min_samples_split=2  )
rf_tuned_v3.fit(X_train, y.to_numpy())
true_y_pred_TEST_rf_tuned = rf_tuned_v3.predict(real_X_test)

In [51]:
true_y_pred_TEST_rf_tuned

array([0.46401508, 0.44816158, 0.47457825, ..., 0.55874871, 0.56305854,
       0.64304008])

### Creating our submission

In [52]:
submission = pd.DataFrame.from_dict({'Completion_rate': true_y_pred_TEST_rf_tuned})
submission

,Completion_rate
0,0.464015
1,0.448162
2,0.474578
3,0.434783
4,0.485761
...,...
1160,0.465618
1161,0.494526
1162,0.558749
1163,0.563059


In [53]:
submission.to_csv('submission.csv', index = True, index_label = "id") 

### To submit your submission

On the panel on your right, select the drop down "Submit to competition", and submit this notebook. Ensure that your submission is named ```submission.csv```!